In [ ]:
import ccxt
import time
import pandas as pd
# Defining assets
CRYPTO_LIST = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'SOL/USDT', 'BUSD/USDT'] #need from 20201017 have existed 

FREQ = '4h'
LIMIT = 1000
FOLDER = ''
API_KEY = '' #change api
SECRET_KEY = ''

ohlcvs = []

exchange = ccxt.binance({"apiKey": API_KEY, "secret": SECRET_KEY, 'enableRateLimit': True})

START_TIMES = [exchange.parse8601('2020-10-17T08:00:00Z'),exchange.parse8601('2021-04-02T00:00:00Z'),
               exchange.parse8601('2021-09-15T16:00:00Z'),exchange.parse8601('2022-03-01T08:00:00Z'),
               exchange.parse8601('2022-08-15T00:00:00Z'),exchange.parse8601('2023-01-28T16:00:00Z'),
               exchange.parse8601('2023-07-14T08:00:00Z')] #6 months add ,exchange.parse8601('2024-01-11T08:00:00Z')

def get_ready_data(freq, crypto_list, limit, start_time):

    dfs = []

    for i in range(len(crypto_list)):

        TICKER = crypto_list[i]

        try:
            bars = exchange.fetch_ohlcv(TICKER, timeframe=freq, limit=limit, since=start_time)
            df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['ticker'] = TICKER
            dfs.append(df)
            time.sleep(1)

        except Exception as e:
            print(f"ticker {TICKER} : {str(e)}")

    ohlcv_df = pd.concat(dfs)
    ohlcv_df = ohlcv_df[['timestamp', 'ticker', 'close', 'open', 'high', 'low', 'volume']]
    ohlcv_df.columns = ['date', 'ticker', 'close', 'open', 'high', 'low', 'volume']
    ohlcv_df['date'] = pd.to_datetime(ohlcv_df['date'], unit='ms')
    ohlcv_df = ohlcv_df.set_index(["date", "ticker"]).sort_index().reset_index()

    return ohlcv_df

for t in START_TIMES:
    data_kwargs = {
        'freq': FREQ,
        'crypto_list': CRYPTO_LIST,
        'limit': LIMIT,
        'start_time': t
    }

    ohlcv = get_ready_data(**data_kwargs)
    ohlcv['date'] = ohlcv['date'].apply(lambda x:str(x))
    ohlcvs.append(ohlcv)

ohlcv = pd.concat(ohlcvs).reset_index(drop=True).set_index(['date','ticker']).sort_index().reset_index()
ohlcv.to_csv(f'{FOLDER}ohlcv.csv', index=False)
ohlcv

Exception ignored in: <function tqdm.__del__ at 0x000001E13C4B19D0>
Traceback (most recent call last):
  File "C:\Users\853uj\anaconda3\lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "C:\Users\853uj\anaconda3\lib\site-packages\tqdm\notebook.py", line 288, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


,date,ticker,close,open,high,low,volume
0,2020-10-17 08:00:00,BNB/USDT,30.4400,30.2351,30.5640,30.1563,2.966816e+05
1,2020-10-17 08:00:00,BTC/USDT,11343.1300,11344.9900,11358.8800,11308.6300,3.658009e+03
2,2020-10-17 08:00:00,BUSD/USDT,0.9996,0.9994,0.9996,0.9994,6.915902e+06
3,2020-10-17 08:00:00,ETH/USDT,368.1200,367.7900,368.4700,366.2900,3.795158e+04
4,2020-10-17 08:00:00,SOL/USDT,2.2145,2.2241,2.2462,2.2025,7.697393e+04
...,...,...,...,...,...,...,...
33475,2023-11-07 04:00:00,BNB/USDT,252.3000,251.5000,252.6000,250.6000,3.433592e+04
33476,2023-11-07 04:00:00,BTC/USDT,34902.8100,34919.0100,34941.5700,34822.3600,1.463282e+03
33477,2023-11-07 04:00:00,BUSD/USDT,0.9999,0.9999,1.0000,0.9999,2.651037e+06
33478,2023-11-07 04:00:00,ETH/USDT,1892.1500,1890.1100,1896.5200,1885.5400,1.594538e+04


In [ ]:
ohlcv['ticker'].value_counts()

BNB/USDT     6618
BTC/USDT     6618
BUSD/USDT    6618
ETH/USDT     6618
SOL/USDT     6618
Name: ticker, dtype: int64